# FlowerBot - Custom Vision Model Training and Deployment

#### Ensure Azure Custom Vision python SDK is installed

In [7]:
!pip install azure-cognitiveservices-vision-customvision

In [8]:
import os
import time
import uuid

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#### Define identifiers used to access Azure Custom Vision services

In [9]:
TRAIN_ENDPOINT = "https://flowerbot002.cognitiveservices.azure.com/"
training_key = "1d95ce065a9143ceb60056449993147a"
PREDICT_ENDPOINT = "https://flowerbot002-prediction.cognitiveservices.azure.com/"
prediction_key = "395816cd19174d6a91be4ee588053fee"
prediction_resource_id = "/subscriptions/55f090ce-963c-4515-a48f-58da5dc82243/resourceGroups/flowerbot/providers/Microsoft.CognitiveServices/accounts/flowerbot-Prediction"

#### Define variables used to access Azure Custom Vision services

In [10]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAIN_ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICT_ENDPOINT, prediction_credentials)

#### Create a new Custom Vision project

In [11]:
# Find the object detection domain
object_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Append a UUID to avoid project name collisions.
publish_iteration_name = "flowerbot-" + str(uuid.uuid4())

print("Creating project", publish_iteration_name)
project = trainer.create_project(publish_iteration_name, domain_id=object_detection_domain.id)

Creating project flowerbot-8be0259d-9193-4fb5-a902-10ef62840711


In [39]:
print(project.id)

5478e771-8f36-4b82-a455-9bc728158288


#### Register tags for the Custom Vision project

In [12]:
tag_names = [
    'astilbe',
    'bellflower',
    'black_eyed_susan',
    'calendula',
    'california_poppy',
    'carnation',
    'common_daisy',
    'coreopsis',
    'daffodil',
    'dandelion',
    'iris',
    'magnolia',
    'rose',
    'sunflower',
    'tulip',
    'water_lily'
]

In [13]:
tag_id = {}
for tag in tag_names:
    tag_id[tag] = trainer.create_tag(project.id, tag)

#### Upload tagged JPEG images

The dataset being used is: https://www.kaggle.com/datasets/l3llff/flowers

In [14]:
IMAGE_PATH = '../flowers/'
IMAGE_MAX = 100 # images per tag

In [15]:
from os import walk

images = {}
for flower in tag_names:
    path = IMAGE_PATH + flower
    for (_, dirnames, filenames) in walk(path):
        images[flower] = filenames[0:IMAGE_MAX]

#### Upload the files to the project, associating each with a tag

These form the basis of our training data and associated the files with tags.

In [16]:
def upload_batch(images, count, total):
    print('Uploading %d images (%d/%d)' % (len(images), count, total))
    batch = ImageFileCreateBatch(images=images)
    result = trainer.create_images_from_files(project.id, batch)
    if result.is_batch_successful:
        print('Upload completed.')
    else:
        print("Image batch upload failed.")
        for image in result.images:
            print("Image status: ", image.status)
        exit(-1)

In [ ]:
batch_size = 64
complete = False
count = 0
total = len(tag_names) * IMAGE_MAX

print("Adding", total, "images in batches of", batch_size)

tagged_images = []
for flower in images.keys():
    for imagefile in images[flower]:
        filename = IMAGE_PATH + flower + '/' + imagefile
        with open(filename, mode="rb") as image:
            tagid = [tag_id[flower].id]
            entry = ImageFileCreateEntry(name=imagefile, contents=image.read(), tag_ids=tagid)
            tagged_images.append(entry)
        count += 1
        if count % batch_size == 0:
            upload_batch(tagged_images, count, total)
            tagged_images.clear()

if len(tagged_images) > 0:
    upload_batch(tagged_images, count, total)

#### Train an iteration of the project, creating a new prediction model

In [ ]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(60)

#### The iteration is now trained, publish it to the project endpoint

In [51]:
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Test the Image Classification Model

In [36]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

In [37]:
PREDICT_ENDPOINT = "https://flowerbot002-prediction.cognitiveservices.azure.com/"
prediction_key = "395816cd19174d6a91be4ee588053fee"
prediction_resource_id = "/subscriptions/55f090ce-963c-4515-a48f-58da5dc82243/resourceGroups/flowerbot/providers/Microsoft.CognitiveServices/accounts/flowerbot-Prediction"

In [38]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICT_ENDPOINT, prediction_credentials)

#### Test the prediction service using an image file

In [ ]:
import os
import time

project_id = '5478e771-8f36-4b82-a455-9bc728158288'
publish_iteration_name = 'flowerbot-8be0259d-9193-4fb5-a902-10ef62840711'
print('Project', project_id, 'iteration', publish_iteration_name)

# Open the sample image and get back the prediction results.
with open(os.path.join("images", "rose.jpg"), mode="rb") as test_data:
    t0 = time.time()
    results = predictor.detect_image(project_id, publish_iteration_name, test_data)
    t1 = time.time()
    print('Prediction elapsed time:', t1 - t0)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name)

## Test the Recommendation Service Model

See also the data/ directory scripts which create and train this model using the command line.

In [6]:
import os
import json
from urllib import parse, request

SERVICE="flowerbotyrpafrhf6sreews"
APIKEY="YWs2NXp1ZTRqaXh3aQ=="
MODEL="ebd580dc-6954-4fe0-ac13-fdc45bb22f88"
ITEM="P00013"

url = 'https://' + SERVICE + '.azurewebsites.net/api/models/' + MODEL + '/recommend'
post = { 'itemId': ITEM }

data = parse.urlencode(post).encode()
req  = request.Request(url, data=data) # this will make the method "POST"
req.add_header('Content-Type', 'application/json')
req.add_header('x-api-key', APIKEY)

resp = request.urlopen(req)
content = resp.read()
print(content)

b'[{"recommendedItemId":"p00011","score":0.0},{"recommendedItemId":"p00015","score":0.0},{"recommendedItemId":"p00013","score":0.0},{"recommendedItemId":"p00012","score":0.0},{"recommendedItemId":"p00006","score":0.0},{"recommendedItemId":"p00009","score":0.0},{"recommendedItemId":"p00014","score":0.0},{"recommendedItemId":"p00002","score":0.0},{"recommendedItemId":"p00008","score":0.0},{"recommendedItemId":"p00004","score":0.0}]'
